In [ ]:
# Using eedomus data for science exploration
# Rain analysis in 2024 - after a high rain season
# You should have a rain sensor with history on eedomus

import requests
import json
import datetime

import matplotlib.pyplot as plt
from google.colab import userdata

##Fetch the entire history
##the secret urljsonrain http://api.eedomus.com/get?api_user=*******&api_secret=*******&action=periph.history&periph_id=9999
urljson = userdata.get('urljsonrain')
urljson = urljson+"&format=json&start_date=&start_date=2024-01-01%2000:00:00"
response = json.loads(requests.get(urljson).text)
data = response["body"]["history"]


serie=[]
ttime=[] #in minute timestamp tz
for items in data:
    serie.append((float(items[0])))
    tstamp=round(datetime.datetime.strptime(items[1], "%Y-%m-%d %H:%M:%S").timestamp())
    ttime.append(tstamp)

plt.plot(ttime, serie)
plt.show()
#beware the order of ttime is anti-chronological



In [ ]:
## Convert the full list into a daily list

#old time, oldts timestamp, old0 start of day
old = ttime[-1]
oldts = datetime.datetime.fromtimestamp(old)
old0 = oldts.replace(hour=0, minute=0, second=0, microsecond=0)

#Calculate the daily amount. First step : builging dailytime serie.
dailytime=[]
dailytime.append(old0)
for index in range(len(ttime)-1, 0, -1):
      indexts = datetime.datetime.fromtimestamp(ttime[index])
      indexts0 = indexts.replace(hour=0, minute=0, second=0, microsecond=0)
      if (indexts0.timestamp() > dailytime[-1].timestamp()):
        dailytime.append(indexts0)

#adding raw cumulative values in dailyseries (without subtracting per day)
subindex = 0
newvalue = True
dailyserie=[]
#dailyserie.append(serie[-1])
#below check last value 0 or -1
for index in range(len(ttime)-1, -1, -1):
      indexts = datetime.datetime.fromtimestamp(ttime[index])
      indexts0 = indexts.replace(hour=0, minute=0, second=0, microsecond=0)
      if (indexts0.timestamp() > dailytime[subindex].timestamp()):
        subindex += 1
        newvalue = True
      else:
        nothing = 0
      if newvalue :
          dailyserie.append(serie[index])
          newvalue=False

## Convert the daily cumulative values into a daily differential value
dailyrain=[]
for i in range (len(dailyserie)-1):
  dailyrain.append(dailyserie[i+1]-dailyserie[i])
  if dailyrain[-1]<0:
      dailyrain[-1]=0
      print("counter reset")

#we add 0 on last value
dailyrain.append(0)

# Montana coefficient gives for the (dep77) location
# rain mm (for 24H) = 8.26 * 1440 min ** (1 - 0.771) # happens every 5 years
montana = 8.26*1440**(1-0.771)
print(montana, " mm in 24H, every 5 years")

plt.title('Daily Rain 2024')
plt.ylabel('mm')
plt.scatter(dailytime, dailyrain)
plt.axhline(y=montana, color='r', linestyle='--')
plt.tight_layout()
plt.show()


![High daily rain](rain_dailyabove5yearsfrequency.png)
As a conclusion, 2 points above the maximum rain every 5 years (nearly 3 points the same year).
The Montana coefficient, calculated on 20 years or more, are probably not accurate any more.